思路是：
1. 因为API需要获得歌曲ID，所以，先根据歌手名，找到歌曲列表，获得歌曲ID。
2. 根据歌曲ID抓取评论
3. 根据抓取评论中的用户ID抓取用户信息

PDF version does not support the Chinese. So, if you can't open the .ipynb file, please see my [Github](https://github.com/YizhuoLi/NetCloudAnalysis).

以下大部分代码参考[NetCloud](https://github.com/Lyrichu/NetCloud)项目改写完成  

## 获取歌曲ID列表

### 1.1 获取歌手热门歌曲ID

In [25]:
# 默认请求头文件
REQUEST_HEADERS = {
        'Host':"music.163.com",
        'User-Agent':"Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0",
        'Accept-Language':"zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3",
        'Accept-Encoding':"gzip, deflate",
        'Content-Type':"application/x-www-form-urlencoded",
        'Cookie':"_ntes_nnid=754361b04b121e078dee797cdb30e0fd,1486026808627; _ntes_nuid=754361b04b121e078dee797cdb30e0fd; JSESSIONID-WYYY=yfqt9ofhY%5CIYNkXW71TqY5OtSZyjE%2FoswGgtl4dMv3Oa7%5CQ50T%2FVaee%2FMSsCifHE0TGtRMYhSPpr20i%5CRO%2BO%2B9pbbJnrUvGzkibhNqw3Tlgn%5Coil%2FrW7zFZZWSA3K9gD77MPSVH6fnv5hIT8ms70MNB3CxK5r3ecj3tFMlWFbFOZmGw%5C%3A1490677541180; _iuqxldmzr_=32; vjuids=c8ca7976.15a029d006a.0.51373751e63af8; vjlast=1486102528.1490172479.21; __gads=ID=a9eed5e3cae4d252:T=1486102537:S=ALNI_Mb5XX2vlkjsiU5cIy91-ToUDoFxIw; vinfo_n_f_l_n3=411a2def7f75a62e.1.1.1486349441669.1486349607905.1490173828142; P_INFO=m15527594439@163.com|1489375076|1|study|00&99|null&null&null#hub&420100#10#0#0|155439&1|study_client|15527594439@163.com; NTES_CMT_USER_INFO=84794134%7Cm155****4439%7Chttps%3A%2F%2Fsimg.ws.126.net%2Fe%2Fimg5.cache.netease.com%2Ftie%2Fimages%2Fyun%2Fphoto_default_62.png.39x39.100.jpg%7Cfalse%7CbTE1NTI3NTk0NDM5QDE2My5jb20%3D; usertrack=c+5+hljHgU0T1FDmA66MAg==; Province=027; City=027; _ga=GA1.2.1549851014.1489469781; __utma=94650624.1549851014.1489469781.1490664577.1490672820.8; __utmc=94650624; __utmz=94650624.1490661822.6.2.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; playerid=81568911; __utmb=94650624.23.10.1490672820",
        'Connection':"keep-alive",
        'Referer':'http://music.163.com/',
        'Upgrade-Insecure-Requests':"1"
        }
# 代理
PROXIES = {
        'http:':'http://122.114.31.177',
        'https:':'https://39.86.40.74'
    }

In [33]:
import requests
import re

def get_singer_hot_songs_ids(singer_url):
    '''
    获取歌手全部id list
    :param singer_url: 歌手主页url
    '''
    ids_list = []
    html = requests.get(
        singer_url,
        headers = REQUEST_HEADERS,
        proxies = PROXIES).text
    pattern = re.compile(r'<a href="/song\?id=(\d+?)">.*?</a>')
    ids = re.findall(pattern,html)
    for id in ids:
        ids_list.append(id)
    return ids_list

### 1.2 获取singer_id与singer_url

In [23]:
MUSIC163_BASE_URL = "http://music.163.com"
SEARCH = '/api/search/get/'

def search(keyword,type_=1, offset=0, limit=30):
    """
    搜索接口,支持搜索音乐,专辑,用户等等
    :param keyword: 搜索关键词
    :param type_: (可选) search type各种取值含义为，1: 音乐, 100:歌手, 1000: 歌单, 1002: 用户
    :param offset: (可选) 起始位置
    :param limit: (可选) 最大数量限制
    """
    # 搜索关键词是必填项
    if keyword is None:
        raise ParamsError()
    method = 'SEARCH'
    # 构造请求数据
    data = {
        's': keyword,
        'limit': str(limit),
        'type': str(type_),
        'offset': str(offset)
    }
    _url = MUSIC163_BASE_URL + SEARCH
    
    resp = requests.post(_url, headers=REQUEST_HEADERS, data=data)
    
    return resp

In [35]:
def get_singer_id_by_name(singer_name,choose_index = 0):
    '''
    通过歌手名字获取歌手id
    :param singer_name: 歌手名
    :param choose_index:选择结果列表第几个结果
    :return:
    '''
    res = search(singer_name,type_ = 100).json()
    singer_id = res['result']['artists'][choose_index]["id"]
    return singer_id

In [30]:
singer_name = '邓紫棋'
singer_id = get_singer_id_by_name(singer_name)

singer_url = 'http://music.163.com/artist?id={singer_id}'.format(singer_id = singer_id)

In [31]:
singer_url

'http://music.163.com/artist?id=7763'

In [36]:
ids_list = get_singer_hot_songs_ids(singer_url)
ids_list[:5]

['449818741', '1299550532', '233974', '36024806', '26113988']

## 根据歌曲ID抓评论

In [146]:
COMMENT = '/api/v1/resource/comments/R_SO_4_'
COMMENTS_NUM_PER_PAGE = 100

def get_current_page_comment(music_id, offset=0):
    
    _url = MUSIC163_BASE_URL + COMMENT + str(music_id) + '?limit='+str(COMMENTS_NUM_PER_PAGE)+'&offset='+str(offset)
    resp = requests.get(_url, headers=REQUEST_HEADERS)
    return resp

In [147]:
def get_song_total_comments_num_and_page_num(resp):
    '''
    获取歌曲评论总数以及评论总页数
    :return:(评论总数,页数)
    '''
    total_comments_num = int(resp.json()['total'])  # 评论总数
    # 获取总页数
    if total_comments_num % COMMENTS_NUM_PER_PAGE == 0:
        page = total_comments_num // COMMENTS_NUM_PER_PAGE
    else:
        page = int(total_comments_num / COMMENTS_NUM_PER_PAGE) + 1
    return total_comments_num,page

In [ ]:
# 服务返回json结果的一些key(str) 映射成常量的形式
HOT_COMMENTS_KEY = "hotComments" # 热评
COMMENTS_KEY = "comments" # 评论
TOTAL_COMMENTS_NUM = "total" # 全部评论数量
COMMENT_CONTENT_KEY = "content" # 评论内容
LIKED_COUNT_KEY = "likedCount" # 赞同数量(点赞数量)
CREATE_TIME_STAMP_KEY = "time" # 评论创建时间戳
USER_KEY = "user" # 用户信息
USER_ID_KEY = "userId" # 用户Id
NICK_NAME_KEY = "nickname" # 用户昵称
AVATAR_URL_KEY = "avatarUrl" # 用户头像地址

In [ ]:
import json

def extract_comment_info_as_json_str(raw_comment):
    '''
    将原始的评论json dict(嵌套) 转化为 自定义形式的简单json str
    :param raw_comment: 原始评论json dict
    :return: converted json str
    ''' 
    comment_dict = {}
    # 评论内容
    comment_dict[COMMENT_CONTENT_KEY] = raw_comment[COMMENT_CONTENT_KEY]
    # 赞同数量
    comment_dict[LIKED_COUNT_KEY] = raw_comment[LIKED_COUNT_KEY]
    # 评论创建时间戳
    comment_dict[CREATE_TIME_STAMP_KEY] = raw_comment[CREATE_TIME_STAMP_KEY]
    # 用户id
    comment_dict[USER_ID_KEY] = raw_comment[USER_KEY][USER_ID_KEY]
    # 用户昵称
    comment_dict[NICK_NAME_KEY] = raw_comment[USER_KEY][NICK_NAME_KEY]
    # dict to str
    return comment_dict
#     return json.dumps(comment_dict,ensure_ascii=False)

In [150]:
# 查看进度
from tqdm import tqdm

def get_song_total_comment(music_id):
    '''
    得到一首歌全部的评论
    :param music_id: 歌曲ID
    :return: 歌曲的评论list
    '''     
    resp = get_current_page_comment(music_id)
    total_comments_num,page = get_song_total_comments_num_and_page_num(resp)
    
    comment_list = []
    
    for i in tqdm(range(page)):
        resp = get_current_page_comment(music_id,i)
        
        resp_json = resp.json()
        for n in range(COMMENTS_NUM_PER_PAGE):
            resp_json_comment = resp_json[COMMENTS_KEY][n]
            comment_list.append(extract_comment_info_as_json_str(resp_json_comment))
            
    return comment_list

In [151]:
def get_singer_total_comment(singer):
    '''
    得到歌手全部的评论，通过遍历歌曲ID
    :param singer: str 歌手名称
    :return: 歌手全部评论评论的list
    '''     
    singer_name = singer
    singer_id = get_singer_id_by_name(singer_name)
    singer_url = 'http://music.163.com/artist?id={singer_id}'.format(singer_id = singer_id)
    ids_list = get_singer_hot_songs_ids(singer_url)
    
    total_comment_list = []
    
    for ids in ids_list:
        comment_list = get_song_total_comment(ids)
        total_comment_list = total_comment_list+comment_list
    return total_comment_list

In [152]:
# 只用了一首歌10页评论做分析，共50*100*10=50000条，因为代码效率不高，跑得太慢
dzq_comment_list = get_singer_total_comment('邓紫棋')


  0%|          | 0/10 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:07<00:00,  1.34it/s]

In [216]:
dzq_comment_list[:5]

[{'content': '如果我没有后顾之忧，父母都身体健康，我应该会跟你走的吧',
  'likedCount': 0,
  'time': 1555011543755,
  'userId': 420887615,
  'nickname': '鱈媛媛'},
 {'content': '不回家',
  'likedCount': 0,
  'time': 1555011332061,
  'userId': 1800398645,
  'nickname': '还没有疯_'},
 {'content': '乎乎',
  'likedCount': 0,
  'time': 1555011323112,
  'userId': 1800398645,
  'nickname': '还没有疯_'},
 {'content': '我没想到为了琳我能疯狂到',
  'likedCount': 0,
  'time': 1555009742039,
  'userId': 568218401,
  'nickname': 'Chen_灿'},
 {'content': '第一遍，没感觉\n第二遍，嗯⊙∀⊙！\n第三遍，真香[流泪]',
  'likedCount': 0,
  'time': 1555008971643,
  'userId': 318682366,
  'nickname': '玛丽到底苏不苏'}]

In [153]:
import pandas as pd

def list_to_df(comment_list):
    '''
    将评论list转化为dataframe方便查看与可视化
    :param singer: str 歌手名称
    :return: 歌手全部评论评论的list
    '''     
    comment_json = json.dumps(comment_list,ensure_ascii=False)
    df = pd.read_json(comment_json, orient='records')
    return df

In [154]:
dzq_comment_df = list_to_df(dzq_comment_list)
dzq_comment_df.sample(10)

,content,likedCount,nickname,time,userId
28584,我知道你很忙，半夜两三点才会跟我说一声晚安。\n我知道你很忙，忙到我在你身边的时候你也是忙不...,2,见你一次就多喜欢一次,1553506579290,103868400
4590,嗯，我会的，谢谢,0,嗨_我姓刘鸭,1554818983801,528408256
40236,08年暑假开始北京三区cf，第一个角色49.9买的飞虎队，那时候我的最爱AN94，生化卡bu...,4,小浪是路人,1554677500229,303093544
17724,他也没杠啊,0,飞行棋mmm,1554775494826,641888970
33991,！！！！！过去的就过去了，再怎么难过也没用！！看明天！,0,沂雨木木,1553091202855,332799019
18083,蕊希推荐❤️,12,蠢蠢Sup,1554387436947,1608499524
6329,不要哭,1,山花cn,1554870672644,572175899
3556,上午没课,0,知日白水,1554949552169,632796653
16415,就是眼鼻嘴啊，翻唱,2,飞行棋mmm,1554773733017,641888970
29420,我是今天的第一个评论✌🏻,1,爱吃猫的鱼miracle,1553729893932,343736492


In [155]:
dzq_comment_df.to_csv('dzq_comment_sample.tsv',sep='\t')

## 获取用户信息

In [198]:
len(dzq_comment_df['userId'])

50000

In [167]:
import time

def from_timestamp_to_date(time_stamp,format = "%Y-%m-%d %H:%M:%S"):
    '''
    将时间戳转化为日期格式
    :param time_stamp: 时间戳
    :param format:指定日期格式
    '''
    real_date = time.strftime(format,time.localtime(time_stamp))
    return real_date

In [190]:
UNKNOWN_TOKEN = "unknown" # 标记未知的标识符
# 服务返回json结果的一些key(str) 映射成常量的形式
HOT_COMMENTS_KEY = "hotComments" # 热评
COMMENTS_KEY = "comments" # 评论
TOTAL_COMMENTS_NUM = "total" # 全部评论数量
COMMENT_CONTENT_KEY = "content" # 评论内容
LIKED_COUNT_KEY = "likedCount" # 赞同数量(点赞数量)
CREATE_TIME_STAMP_KEY = "time" # 评论创建时间戳
USER_KEY = "user" # 用户信息
USER_ID_KEY = "userId" # 用户Id
NICK_NAME_KEY = "nickname" # 用户昵称
AVATAR_URL_KEY = "avatarUrl" # 用户头像地址

GENDER = 'gender'# 性别
PLAYLISTCOUNT = 'playlistCount'# 播放列表数  playlistCount
CREATEDAYS = 'createDays'# 账号创建天数 createDays
LEVEL = 'level'# 用户账号等级
CRAWLER_TIME_KEY = "crawler_time" # 抓取时间
EVENT_COUNT_KEY = "event_count" # 动态总数
FOLLOW_COUNT_KEY = "follow_count" # 用户关注总数
FAN_COUNT_KEY = "fan_count" # 用户粉丝总数
LOCATION_KEY = "location" # 用户所在地区，省份代号
USER_DESCRIPTION_KEY = "user_description" # 用户个人描述
USER_AGE_KEY = "age" # 用户年龄
LISTENING_SONGS_NUM_KEY = "listening_songs_num" # 用户累计听歌数目

In [217]:
USER = '/api/v1/user/detail/'

def get_user_info(user_id):
    
    _url = MUSIC163_BASE_URL + USER + str(user_id)
    user_id = user_id
    # 获取html
    html = requests.get(_url, headers=REQUEST_HEADERS).text
    # 性别
    gender_pattern = re.compile(r'"gender":(\d+?)')
    gender_count = re.search(gender_pattern, html)
    if gender_count:
        gender_count = gender_count.group(1)
    else:
        gender_count = UNKNOWN_TOKEN
    # 播放列表数  playlistCount
    playlistCount_count_pattern = re.compile(r'"playlistCount":(\d+)')
    playlistCount_count = re.search(playlistCount_count_pattern, html)
    if playlistCount_count:
        playlistCount_count = playlistCount_count.group(1)
    else:
        playlistCount_count = UNKNOWN_TOKEN
    # 账号创建天数 createDays
    createDays_count_pattern = re.compile(r'"createDays":(\d+)')
    createDays_count = re.search(createDays_count_pattern, html)
    if createDays_count:
        createDays_count = createDays_count.group(1)
    else:
        createDays_count = UNKNOWN_TOKEN
    # 用户账号等级
    level_count_pattern = re.compile(r'"level":(\d+)')
    level_count = re.search(level_count_pattern, html)
    if level_count:
        level_count = level_count.group(1)
    else:
        level_count = UNKNOWN_TOKEN
    # 动态总数
    event_count_pattern = re.compile(r'"eventCount":(\d+?)')
    event_count = re.search(event_count_pattern, html)
    if event_count:
        event_count = event_count.group(1)
    else:
        event_count = UNKNOWN_TOKEN
    # 用户关注数
    follow_count_pattern = re.compile(r'"follows":(\d+?)')
    follow_count = re.search(follow_count_pattern, html)
    if follow_count:
        follow_count = follow_count.group(1)
    else:
        follow_count = UNKNOWN_TOKEN
    # 用户粉丝数
    fan_count_pattern = re.compile(r'"followeds":(\d+?)')
    fan_count = re.search(fan_count_pattern, html)
    if fan_count:
        fan_count = fan_count.group(1)
    else:
        fan_count = UNKNOWN_TOKEN
    # 用户所在省份代号
    location_pattern = re.compile('"province":(\d+?)')
    location = re.search(location_pattern, html)
    if location:
        location = location.group(1)
    else:
        location = UNKNOWN_TOKEN
    # 用户个人描述
    description_pattern = re.compile('"signature":"(.*?)"')
    description = re.search(description_pattern, html)
    if description:
        description = description.group(1)
    else:
        description = UNKNOWN_TOKEN
    # 用户年龄
    age_pattern = re.compile(r'"birthday":(\d+)')
    age = re.search(age_pattern, html)
    if age:
        age = age.group(1)  # 时间戳形式
        # 今年
        current_year = int(from_timestamp_to_date(time_stamp=time.time(), format="%Y"))
        # 得到用户真实年龄
        age = (current_year - 1970) - int(age) // (1000 * 365 * 24 * 3600)
    else:
        age = UNKNOWN_TOKEN
    # 累计听歌
    listening_songs_num_pattern = re.compile('"listenSongs":(\d+)')
    listening_songs_num = re.search(listening_songs_num_pattern, html)
    if listening_songs_num:
        listening_songs_num = listening_songs_num.group(1)
    else:
        listening_songs_num = UNKNOWN_TOKEN
    # 将用户信息以json形式保存到磁盘
    user_info_dict = {
        GENDER: gender_count,
        PLAYLISTCOUNT: playlistCount_count,
        CREATEDAYS: createDays_count,
        LEVEL: level_count,
        USER_ID_KEY: user_id,
        EVENT_COUNT_KEY: event_count,
        FOLLOW_COUNT_KEY: follow_count,
        FAN_COUNT_KEY: fan_count,
        LOCATION_KEY: location,
        USER_DESCRIPTION_KEY: description,
        USER_AGE_KEY: age,
        LISTENING_SONGS_NUM_KEY: listening_songs_num
    }
#     user_info_json_str = json.dumps(user_info_dict, ensure_ascii=False)
    return user_info_dict
#     return user_info_json_str
#     users_info_list.append(user_info_json_str)

In [218]:
get_user_info(408146618)

{'gender': '1',
 'playlistCount': '10',
 'createDays': '808',
 'level': '8',
 'userId': 408146618,
 'event_count': '0',
 'follow_count': '2',
 'fan_count': '1',
 'location': '4',
 'user_description': '人丑嘴不甜',
 'age': 22,
 'listening_songs_num': '3217'}

In [219]:
def get_all_user_info(user_list):
    
    users_info_list = []
    for user_id in tqdm(user_list):
        user_info_json_dict = get_user_info(user_id)
        users_info_list.append(user_info_json_dict)
    return users_info_list

In [226]:
dzq_fun_list = get_all_user_info(dzq_comment_df['userId'][:1000])





  0%|          | 0/1000 [00:00<?, ?it/s]



  0%|          | 1/1000 [00:00<10:33,  1.58it/s]



  0%|          | 2/1000 [00:01<13:24,  1.24it/s]



  0%|          | 3/1000 [00:02<12:41,  1.31it/s]



  0%|          | 4/1000 [00:03<12:21,  1.34it/s]



  0%|          | 5/1000 [00:03<11:43,  1.41it/s]



  1%|          | 6/1000 [00:04<11:57,  1.38it/s]



  1%|          | 7/1000 [00:05<12:19,  1.34it/s]



  1%|          | 8/1000 [00:05<11:39,  1.42it/s]



  1%|          | 9/1000 [00:06<10:50,  1.52it/s]



  1%|          | 10/1000 [00:07<10:54,  1.51it/s]



  1%|          | 11/1000 [00:07<10:19,  1.60it/s]



  1%|          | 12/1000 [00:08<10:53,  1.51it/s]



  1%|▏         | 13/1000 [00:09<10:09,  1.62it/s]



  1%|▏         | 14/1000 [00:09<09:37,  1.71it/s]



  2%|▏         | 15/1000 [00:10<09:20,  1.76it/s]



  2%|▏         | 16/1000 [00:10<09:10,  1.79it/s]



  2%|▏         | 17/1000 [00:11<09:00,  1.82it/s]



  2%|▏         | 18/1000 [00:11<08:54,  1.84it/s]



  2%|▏ 

 25%|██▍       | 249/1000 [02:17<06:32,  1.92it/s]



 25%|██▌       | 250/1000 [02:18<06:33,  1.90it/s]



 25%|██▌       | 251/1000 [02:18<06:31,  1.91it/s]



 25%|██▌       | 252/1000 [02:19<06:30,  1.92it/s]



 25%|██▌       | 253/1000 [02:19<06:30,  1.91it/s]



 25%|██▌       | 254/1000 [02:20<06:47,  1.83it/s]



 26%|██▌       | 255/1000 [02:20<06:40,  1.86it/s]



 26%|██▌       | 256/1000 [02:21<06:37,  1.87it/s]



 26%|██▌       | 257/1000 [02:21<06:34,  1.88it/s]



 26%|██▌       | 258/1000 [02:22<06:34,  1.88it/s]



 26%|██▌       | 259/1000 [02:22<06:32,  1.89it/s]



 26%|██▌       | 260/1000 [02:23<06:40,  1.85it/s]



 26%|██▌       | 261/1000 [02:24<06:59,  1.76it/s]



 26%|██▌       | 262/1000 [02:24<07:11,  1.71it/s]



 26%|██▋       | 263/1000 [02:25<06:56,  1.77it/s]



 26%|██▋       | 264/1000 [02:25<06:46,  1.81it/s]



 26%|██▋       | 265/1000 [02:26<06:38,  1.84it/s]



 27%|██▋       | 266/1000 [02:26<06:35,  1.86it/s]



 27%|██▋       | 267/1000 [0

 50%|████▉     | 497/1000 [04:29<04:30,  1.86it/s]



 50%|████▉     | 498/1000 [04:30<04:27,  1.88it/s]



 50%|████▉     | 499/1000 [04:30<04:27,  1.87it/s]



 50%|█████     | 500/1000 [04:31<04:34,  1.82it/s]



 50%|█████     | 501/1000 [04:31<04:36,  1.81it/s]



 50%|█████     | 502/1000 [04:32<04:46,  1.74it/s]



 50%|█████     | 503/1000 [04:32<04:38,  1.79it/s]



 50%|█████     | 504/1000 [04:33<04:31,  1.83it/s]



 50%|█████     | 505/1000 [04:33<04:26,  1.86it/s]



 51%|█████     | 506/1000 [04:34<04:22,  1.88it/s]



 51%|█████     | 507/1000 [04:35<04:26,  1.85it/s]



 51%|█████     | 508/1000 [04:35<04:22,  1.87it/s]



 51%|█████     | 509/1000 [04:36<04:20,  1.88it/s]



 51%|█████     | 510/1000 [04:36<04:20,  1.88it/s]



 51%|█████     | 511/1000 [04:37<04:19,  1.88it/s]



 51%|█████     | 512/1000 [04:37<04:18,  1.89it/s]



 51%|█████▏    | 513/1000 [04:38<04:14,  1.91it/s]



 51%|█████▏    | 514/1000 [04:38<04:12,  1.92it/s]



 52%|█████▏    | 515/1000 [0

 74%|███████▍  | 745/1000 [06:43<02:20,  1.82it/s]



 75%|███████▍  | 746/1000 [06:43<02:18,  1.84it/s]



 75%|███████▍  | 747/1000 [06:44<02:19,  1.82it/s]



 75%|███████▍  | 748/1000 [06:45<02:17,  1.84it/s]



 75%|███████▍  | 749/1000 [06:45<02:14,  1.87it/s]



 75%|███████▌  | 750/1000 [06:46<02:12,  1.88it/s]



 75%|███████▌  | 751/1000 [06:46<02:14,  1.85it/s]



 75%|███████▌  | 752/1000 [06:47<02:12,  1.88it/s]



 75%|███████▌  | 753/1000 [06:47<02:10,  1.89it/s]



 75%|███████▌  | 754/1000 [06:48<02:09,  1.90it/s]



 76%|███████▌  | 755/1000 [06:48<02:18,  1.77it/s]



 76%|███████▌  | 756/1000 [06:49<02:14,  1.81it/s]



 76%|███████▌  | 757/1000 [06:49<02:12,  1.84it/s]



 76%|███████▌  | 758/1000 [06:50<02:09,  1.87it/s]



 76%|███████▌  | 759/1000 [06:51<02:15,  1.78it/s]



 76%|███████▌  | 760/1000 [06:51<02:34,  1.55it/s]



 76%|███████▌  | 761/1000 [06:52<02:28,  1.61it/s]



 76%|███████▌  | 762/1000 [06:52<02:20,  1.69it/s]



 76%|███████▋  | 763/1000 [0

 99%|█████████▉| 993/1000 [08:57<00:03,  1.89it/s]



 99%|█████████▉| 994/1000 [08:58<00:03,  1.90it/s]



100%|█████████▉| 995/1000 [08:58<00:02,  1.93it/s]



100%|█████████▉| 996/1000 [08:59<00:02,  1.89it/s]



100%|█████████▉| 997/1000 [08:59<00:01,  1.90it/s]



100%|█████████▉| 998/1000 [09:00<00:01,  1.90it/s]



100%|█████████▉| 999/1000 [09:00<00:00,  1.87it/s]



100%|██████████| 1000/1000 [09:01<00:00,  1.88it/s]

In [227]:
dzq_fun_df = list_to_df(dzq_fun_list)
dzq_fun_df.sample(5)

,age,createDays,event_count,fan_count,follow_count,gender,level,listening_songs_num,location,playlistCount,userId,user_description
429,0,6,0,5,3,0,1,226,4,2,1816955658,
768,19,458,1,5,3,1,7,2956,3,7,1336992762,你可以坐在扶手椅上继续读你的书，种你的树，看着它们长大。
221,unknown,890,0,1,1,1,7,1042,4,6,356267853,
835,20,667,8,6,5,1,7,2063,1,9,507793255,"中考备战,十年磨一剑,至少应该做一回自己的英雄。"
456,unknown,486,0,2,8,2,7,1509,3,3,1309613719,


In [228]:
len(dzq_fun_df)

1000

In [229]:
dzq_fun_df.to_csv('dzq_fun_sample.tsv',sep='\t')